In [5]:
!pip install gym

In [1]:
!pip install tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tools: filename=tools-0.1.9-py3-none-any.whl size=46731 sha256=3b793c4ffddf777a9711997bd5eb2f8aff83dd5fa223d55bf854f8e4e00bab9f
  Stored in directory: /root/.cache/pip/wheels/c0/d0/70/a33bd4bed2af4f7038b038c16faab552cd0e9d9f4125223a71
  Created wheel for pytils: filename=pytils-0.4.1-py3-none-any.whl size=32536 sha256=f0180fb3e8976369d57fb6394f3786803a8f261f52b449fe41dfa276ec876547
  Stored in directory: /root/.cache/pip/wheels/5a/eb/7c/3b6f0c25815749883152b2caca34c35dbaab13ec2864270cbd
Successfully built tools pytils


In [4]:
import gym
import numpy as np
import tools

env = gym.make('FrozenLake-v1')
env.reset()

gamma = 0.99
theta = 0.000001

def argmax(env, V, pi, action,s, gamma):
    e = np.zeros(env.env.nA)
    for a in range(env.env.nA):                         # iterate for every action possible
        q=0
        P = np.array(env.env.P[s][a])
        (x,y) = np.shape(P)                             # for Bellman Equation

        for i in range(x):                              # iterate for every possible states
            s_= int(P[i][1])                            # S' - Sprime - possible succesor states
            p = P[i][0]                                 # Transition Probability P(s'|s,a)
            r = P[i][2]                                 # Reward

            q += p*(r+gamma*V[s_])                      # calculate action_ value q(s|a)
            e[a] = q

    m = np.argmax(e)
    action[s]=m                                           # Take index which has maximum value
    pi[s][m] = 1                                        # update pi(a|s)

    return pi


def bellman_optimality_update(env, V, s, gamma):  # update the stae_value V[s] by taking
    pi = np.zeros((env.env.nS, env.env.nA))       # action which maximizes current value
    e = np.zeros(env.env.nA)
                                            # STEP1: Find
    for a in range(env.env.nA):
        q=0                                 # iterate for all possible action
        P = np.array(env.env.P[s][a])
        (x,y) = np.shape(P)

        for i in range(x):
            s_= int(P[i][1])
            p = P[i][0]
            r = P[i][2]
            q += p*(r+gamma*V[s_])
            e[a] = q

    m = np.argmax(e)
    pi[s][m] = 1

    value = 0
    for a in range(env.env.nA):
        u = 0
        P = np.array(env.env.P[s][a])
        (x,y) = np.shape(P)
        for i in range(x):

            s_= int(P[i][1])
            p = P[i][0]
            r = P[i][2]

            u += p*(r+gamma*V[s_])

        value += pi[s,a] * u

    V[s]=value
    return V[s]



def value_iteration(env, gamma, theta):
    V = np.zeros(env.env.nS)                                       # initialize v(0) to arbitory value, my case "zeros"
    while True:
        delta = 0
        for s in range(env.env.nS):                       # iterate for all states
            v = V[s]
            bellman_optimality_update(env, V, s, gamma)   # update state_value with bellman_optimality_update
            delta = max(delta, abs(v - V[s]))             # assign the change in value per iteration to delta
        if delta < theta:
            break                                         # if change gets to negligible
                                                          # --> converged to optimal value
    pi = np.zeros((env.env.nS, env.env.nA))
    action = np.zeros((env.env.nS))
    for s in range(env.env.nS):
        pi = argmax(env, V, pi,action, s, gamma)         # extract optimal policy using action value

    return V, pi,action                                          # optimal value funtion, optimal policy


V, pi, action = value_iteration(env, gamma, theta)
tools.plot(V,pi)



a= np.reshape(action,(4,4))
print(a)                          # discrete action to take in given state


e=0
for i_episode in range(100):
    c = env.reset()
    for t in range(10000):
        c, reward, done, info = env.step(action[c])
        if done:
            if reward == 1:
                e +=1
            break
print(" agent succeeded to reach goal {} out of 100 Episodes using this policy ".format(e+1))
env.close()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


AttributeError: ignored

In [13]:
""" Solving FrozenLake8x8 from OpenAI using Value Iteration

    Author: Diganta Kalita  (digankate26@gmail.com) """

import gym
import numpy as np


def value_iteration(env, max_iterations=100000, lmbda=0.9):
  stateValue = [0 for i in range(env.observation_space.n)]
  newStateValue = stateValue.copy()
  for i in range(max_iterations):
    for state in range(env.observation_space.n):
      action_values = []
      for action in range(env.action_space.n):
        state_value = 0
        for i in range(len(env.P[state][action])):
          prob, next_state, reward, done = env.P[state][action][i]
          state_action_value = prob * (reward + lmbda*stateValue[next_state])
          state_value += state_action_value
        action_values.append(state_value)      #the value of each action
        best_action = np.argmax(np.asarray(action_values))   # choose the action which gives the maximum value
        newStateValue[state] = action_values[best_action]  #update the value of the state
    if i > 1000:
      if sum(stateValue) - sum(newStateValue) < 1e-04:   # if there is negligible difference break the loop
        break
        print(i)
    else:
      stateValue = newStateValue.copy()
  return stateValue

def get_policy(env,stateValue, lmbda=0.9):
  policy = [0 for i in range(env.observation_space.n)]
  for state in range(env.observation_space.n):
    action_values = []
    for action in range(env.action_space.n):
      action_value = 0
      for i in range(len(env.P[state][action])):
        prob, next_state, r, _ = env.P[state][action][i]
        action_value += prob * (r + lmbda * stateValue[next_state])
      action_values.append(action_value)
    best_action = np.argmax(np.asarray(action_values))
    policy[state] = best_action
  return policy


def get_score(env, policy, episodes=1000):
  misses = 0
  steps_list = []
  for episode in range(episodes):
    observation = env.reset()
    steps=0
    while True:

      action = policy[observation]
      observation, reward, done, _ = env.step(action)
      steps+=1
      if done and reward == 1:
        # print('You have got the fucking Frisbee after {} steps'.format(steps))
        steps_list.append(steps)
        break
      elif done and reward == 0:
        # print("You fell in a hole!")
        misses += 1
        break
  print('----------------------------------------------')
  print('You took an average of {:.0f} steps to get the frisbee'.format(np.mean(steps_list)))
  print('And you fell in the hole {:.2f} % of the times'.format((misses/episodes) * 100))
  print('----------------------------------------------')



env = gym.make('FrozenLake8x8-v1')

stateValues = value_iteration(env, max_iterations=100000)
policy = get_policy(env, stateValues)
get_score(env, policy,episodes=1000)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


----------------------------------------------
You took an average of 72 steps to get the frisbee
And you fell in the hole 27.70 % of the times
----------------------------------------------


In [8]:
env = gym.make('FrozenLake8x8-v1')

In [10]:
env.reset()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0